<a href="https://colab.research.google.com/github/yg20029/AI-X_DeepLearning/blob/main/resnet34.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
#cb8d12c2a852272d7e90082a3e04e2092922d1c3
%%capture
!pip install wandb -qU
import wandb
wandb.login()

In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
import wandb

# WandB 초기화
wandb.init(project="resnet18-training")

# 데이터 준비
data_dir = '/root/.cache/kagglehub/datasets/gpiosenka/sports-classification/versions/9/'
transform = {
    'train': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'valid': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}

train_dataset = datasets.ImageFolder(data_dir + 'train', transform=transform['train'])
valid_dataset = datasets.ImageFolder(data_dir + 'valid', transform=transform['valid'])

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=64, shuffle=False)

# ResNet18 불러오기 및 수정
model = models.resnet18(pretrained=True)

# Fine-tuning: 특정 레이어만 학습 가능하도록 설정
for name, param in model.named_parameters():
    if "layer3" in name or "layer4" in name or "fc" in name:
        param.requires_grad = True
    else:
        param.requires_grad = False

# 출력층 수정
model.fc = nn.Sequential(
    nn.Linear(model.fc.in_features, 512),
    nn.ReLU(),
    nn.Dropout(0.4),
    nn.Linear(512, 100)
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# 손실 함수와 옵티마이저 설정
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=0.00005, weight_decay=0.01)

# WandB에 모델 구조 및 파라미터 정보 저장
wandb.watch(model, log="all")

# 학습 및 검증 결과 저장

def train_model(model, train_loader, valid_loader, criterion, optimizer, epochs=5):
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

    for epoch in range(epochs):
        print(f"Epoch {epoch+1}/{epochs}")
        model.train()
        train_loss, train_correct = 0.0, 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            train_correct += torch.sum(preds == labels.data)

        train_loss /= len(train_loader.dataset)
        train_acc = train_correct.double() / len(train_loader.dataset)

        model.eval()
        valid_loss, valid_correct = 0.0, 0

        with torch.no_grad():
            for inputs, labels in valid_loader:
                inputs, labels = inputs.to(device), labels.to(device)

                outputs = model(inputs)
                loss = criterion(outputs, labels)

                valid_loss += loss.item() * inputs.size(0)
                _, preds = torch.max(outputs, 1)
                valid_correct += torch.sum(preds == labels.data)

        valid_loss /= len(valid_loader.dataset)
        valid_acc = valid_correct.double() / len(valid_loader.dataset)

        print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}")
        print(f"Valid Loss: {valid_loss:.4f}, Valid Acc: {valid_acc:.4f}")

        # 평균 Gradient 크기 계산 및 로그
        avg_gradient = 0
        for param in model.parameters():
            if param.grad is not None:
                avg_gradient += param.grad.abs().mean().item()
        avg_gradient /= len([p for p in model.parameters() if p.grad is not None])

        # WandB 로그 저장
        wandb.log({
            "epoch": epoch + 1,
            "train_loss": train_loss,
            "train_acc": train_acc.item(),
            "valid_loss": valid_loss,
            "valid_acc": valid_acc.item(),
            "learning_rate": optimizer.param_groups[0]['lr'],
            "avg_gradient": avg_gradient
        })

        scheduler.step()

# 학습 실행
train_model(model, train_loader, valid_loader, criterion, optimizer, epochs=10)

# WandB 종료
wandb.finish()
